# 3D Pose Generation

With `NextFace`, we can generate a face with various poses from a single image. This is useful when we want to expand our database, in cases where we only have a single image/subject.

For more details, check out their [repository](https://github.com/abdallahdib/NextFace)!

For this notebook, you need a different working environment from the previous notebooks. For the installation guide, please refer [here](https://github.com/abdallahdib/NextFace/blob/main/INSTALL).

Or, in your Anaconda Prompt, run these commands

    conda create --name faceNext python=3.6.7
    conda activate faceNext
    conda install pytorch==1.3.1 torchvision==0.4.2 cudatoolkit=10.1 -c pytorch
    pip install redner-gpu
    conda install -c conda-forge opencv
    conda install -c 1adrianb face_alignment=1.2.0
    conda install -c anaconda h5py
    pip install mediapipe

In [ ]:
from optimizer import Optimizer
from config import Config
from utils import *
import math
from image import saveImage
import json

In [6]:
from models.NextFace.optimizer import optimizer

ModuleNotFoundError: No module named 'pyredner'